# Data engineering
## From "DE_Task.ipynb" February 7, 2025 
### Task 7
(use venv_requirements.txt)

The hand-in exercise for this topic is Task 7 from the notebook “DE_Task.ipynb”. The
exercise asks you to wrap the functionality of other tasks in this notebook into an ETL
pipeline. Note that the GroupBY tasks (which are extra) are not needed in this ETL pipeline.
The data visualization parts are also not needed. The pipeline should contain some of the
data cleaning (such as removing duplicates and nulls) and then creation of features which
are part of task 4 and 5.

In [ ]:
import pandas as pd

In [ ]:
def extract(dataset_path):
    # An error in the reading of the excel file caused only 1st sheet of the excel file to be accessed.
    # This caused a loss of  the data especially since sheet 1 includes data from '09-'10 and sheet 2 shows data from '10-'11.
    # Specifying sheet_name=None, to return all sheets in a dict(xls)
    # Source: (https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html)
    xls = pd.read_excel(dataset_path, sheet_name=None)
    s1 = xls['Year 2009-2010']
    s2 = xls['Year 2010-2011']
    #combining the two dataframes into one
    df = pd.concat([s1,s2])
    return df

def transform(df):
    
    # Fill missing 'Description' using the most common value per 'StockCode'
    df['Description'] = df.groupby('StockCode')['Description'].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)

    #Drop rows where 'Customer ID' is missing
    df = df.dropna(subset=['Customer ID'])
    # Drop duplicates
    df.drop_duplicates(inplace=True)

    # Adding column TotalSum, to make it easier to calculcate Revenue
    df['TotalSum'] = pd.DataFrame(df['Price'].values  * df['Quantity'].values)
    # Setting the week day and adding it as a column
    df= df.set_index('InvoiceDate').reset_index()
    df['DayOfWeek'] = pd.to_datetime(df['InvoiceDate']).dt.dayofweek


    day_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    #mapping the labels to the numerical values
    df['DayOfWeek'] = df['DayOfWeek'].map(lambda x: day_labels[x])
    df["total_revenue"] = df['Customer ID'].map(df.groupby("Customer ID")["Price"].sum())


    #logic for creating the tiers in the main dataframe
    customer_tier = lambda tier: 'Low' if tier < 139 else ('Medium' if tier <= 337 else 'High') 
    #basing the logic on the temp DF and bringing the result to the main DF
    df['Customer_Tier_Category'] = df['total_revenue'].apply(customer_tier)
    # Extract key information from Description and add them as columns (e.g., presence of specific keywords like "Gift" or "Discount"). At least one extra column should be added
    words=['set', 'assortment', 'series', 'selection']
    # interates over the list words above. matching any occurance of the word in the description (to lower case).
    # sets a true / false value in a column called isACollection  
    df['isACollection'] = df['Description'].str.lower().apply(lambda x: any(word in x for word in words))

    #Feature engineering a new column adding Transaction Tiers
    transaction_tier = lambda tier: 'Low' if tier < 133 else ('Medium' if tier <= 399 else 'High') 
    df['Transaction_Tier_Category'] = df['total_revenue'].apply(transaction_tier)

    #Feature engineering: Festive items
    words  = [ "festive", "merry", "mistletoe", "santa", "carols", "decorations", "eggnog", "elves", "holly", "reindeer", "tinsel", "angel", "evergreen", "snow", "stocking", "suletide"] 
    df['isFestive'] = df['Description'].str.lower().apply(lambda x: any(word in x for word in words))

    #Feature engineering: Customer Loyality tiers   
    loality_tier = lambda count: 'One-Time' if count == 1 else ('Occasional' if count < 51 else 'High') 
    # Count occurrences of each Customer ID
    customer_counts = df['Customer ID'].value_counts().reset_index()
    # Apply the loyalty tier function
    customer_counts['Customer_Loyality'] = customer_counts['count'].apply(loality_tier)
    # Merge back into the original DataFrame
    df = df.merge(customer_counts[['Customer ID', 'Customer_Loyality']], on='Customer ID', how='left')

    #Feature Engineering: Order Type
    order_type = lambda x: 'Muli-item Order' if x > 1 else 'Single-item Order'
    invoice_item_count =  df.groupby('Invoice')['StockCode'].nunique().reset_index()
    invoice_item_count['Order_Type'] =invoice_item_count['StockCode'].apply(order_type)
    df = df.merge(invoice_item_count[['Invoice', 'Order_Type']], on='Invoice', how='left')
    return df

def load(df, output_path):
    #df.to_csv(output_path, index=False)
    print("Data saved")
    return df


def ETL(dataset_path, output_path):
    df= extract(dataset_path)
    df_transformed = transform(df)
    df_loaded = load(df_transformed, output_path)
    return df_loaded


In [5]:
ETL("C:\\Users\\Hassan\\.cache\\kagglehub\\datasets\\lakshmi25npathi\\online-retail-dataset\\versions\\1\\online_retail_II.xlsx", "C:\\Users\\Hassan\\Desktop\\test")

C:\Users\Hassan\AppData\Local\Temp\ipykernel_16448\3745648497.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
C:\Users\Hassan\AppData\Local\Temp\ipykernel_16448\3745648497.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalSum'] = pd.DataFrame(df['Price'].values  * df['Quantity'].values)


Data saved


,InvoiceDate,Invoice,StockCode,Description,Quantity,Price,Customer ID,Country,TotalSum,DayOfWeek,total_revenue,Customer_Tier_Category,isACollection,Transaction_Tier_Category,isFestive,Customer_Loyality,Order_Type
0,2009-12-01 07:45:00,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,13085.0,United Kingdom,83.4,Tuesday,1142.05,High,False,High,False,High,Muli-item Order
1,2009-12-01 07:45:00,489434,79323P,PINK CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,81.0,Tuesday,1142.05,High,False,High,False,High,Muli-item Order
2,2009-12-01 07:45:00,489434,79323W,WHITE CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,81.0,Tuesday,1142.05,High,False,High,False,High,Muli-item Order
3,2009-12-01 07:45:00,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,13085.0,United Kingdom,100.8,Tuesday,1142.05,High,False,High,False,High,Muli-item Order
4,2009-12-01 07:45:00,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,13085.0,United Kingdom,30.0,Tuesday,1142.05,High,False,High,False,High,Muli-item Order
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797880,2011-12-09 12:50:00,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2.10,12680.0,France,207.5,Friday,207.17,Medium,False,Medium,False,High,Muli-item Order
797881,2011-12-09 12:50:00,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,4.15,12680.0,France,20.8,Friday,207.17,Medium,False,Medium,False,High,Muli-item Order
797882,2011-12-09 12:50:00,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,12680.0,France,8.5,Friday,207.17,Medium,False,Medium,False,High,Muli-item Order
797883,2011-12-09 12:50:00,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,4.95,12680.0,France,20.8,Friday,207.17,Medium,True,Medium,False,High,Muli-item Order
